## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
#file_index='BBBSBBBB'
file_index='SBSSBBBS'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
502
+------------------+-------------------+-----------------+---------+--------+--------+---------+-----------+-------------------+--------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|           coeff_1|            coeff_2|          coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|               res_2|              res_3|          res_mean|    station|   total_var|undefs|              vector|  year|
+------------------+-------------------+-----------------+---------+--------+--------+---------+-----------+-------------------+--------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|-8864.636346120038|-2039.9548861316823|698.3849072461769|   1797.4|SBSSBBBS|  43.465|-113.5581|       SNWD| 0.0848993837998154| 0.03534173056103785|0.03086802364758516|0.74896324

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00102260,1993.0,-2039.954886
1,USC00104845,1997.0,-665.958256
2,USC00104845,1980.0,1176.284964
3,USC00104230,1972.0,-1602.792073
4,USC00103942,1952.0,145.355169


In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,USC00100010,USC00100227,USC00100375,USC00100915,USC00102260,USC00102676,USC00102823,USC00103110,USC00103297,USC00103825,...,USC00108380,USC00108535,USC00108906,USC00108937,USC00109601,USC00480124,USW00024142,USW00024145,USW00024156,USW00094143
year,,,,,,,,,,,,,,,,,,,,,
1911.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1949.0,-311.295883,-290.428831,NaN,-207.943600,NaN,NaN,-205.352714,NaN,-194.860696,NaN,...,-316.811916,NaN,-1017.462545,NaN,NaN,NaN,-500.153900,-80.754492,-168.729796,NaN
1950.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-223.361358,...,-310.304384,NaN,-1082.638214,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1133.853458,...,859.979699,NaN,2847.713816,NaN,NaN,NaN,714.876736,NaN,NaN,NaN
1952.0,NaN,NaN,NaN,-177.102987,NaN,NaN,NaN,NaN,NaN,-1120.167539,...,-632.006383,NaN,303.156438,NaN,NaN,NaN,-703.317794,-44.613095,-116.196069,NaN
1953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-283.346101,...,NaN,NaN,-572.117023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1954.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-746.534632,...,NaN,NaN,-1015.076603,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1955.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,294.499732,NaN,450.585265,NaN,NaN,NaN,NaN,-408.470649,NaN,NaN
1956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-404.590029,NaN,NaN,NaN,NaN,-214.897439,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

# Intuition from below output based on snow output is as follows
# Effect of year to year variation is more than the station as RMS is more reduction by removing mean by year

total RMS                   =  848.69917793
RMS removing mean-by-station=  828.268337939
RMS removing mean-by-year   =  540.693118362


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)
    
# Measurements over the years data is much noisy and hence we rely on these PCA components
# 

initial RMS= 848.69917793
0 after removing mean by year    = 540.693118362
0 after removing mean by stations= 526.181161298
1 after removing mean by year    = 524.635943663
1 after removing mean by stations= 524.321748828
2 after removing mean by year    = 524.224364916
2 after removing mean by stations= 524.186093351
3 after removing mean by year    = 524.169321724
3 after removing mean by stations= 524.161576845
4 after removing mean by year    = 524.157893966
4 after removing mean by stations= 524.156109745
